In [26]:
import json
import os
from typing import List

dataset = "webqsp" # or "cwq"

# rog_rules
rog_rules_dir = os.path.join(os.path.dirname("__file__"), 'gen_rule_path', f'RoG-{dataset}', 'RoG', 'test')
rog_rules_file = os.path.join(rog_rules_dir, 'predictions_3_False.jsonl')

# rog_reasoning
rog_reasoning_dir = os.path.join(os.path.dirname("__file__"), 'KGQA', f'RoG-{dataset}', 'RoG', 'test', f'results_gen_rule_path_RoG-{dataset}_RoG_test_predictions_3_False_jsonl')
rog_reasoning_questions = os.path.join(rog_reasoning_dir, 'predictions.jsonl')
rog_reasoning_predictions = os.path.join(rog_reasoning_dir, 'detailed_eval_result.jsonl')

In [27]:
rog_rules_file

'gen_rule_path/RoG-webqsp/RoG/test/predictions_3_False.jsonl'

In [66]:
def get_error_flag(predictions: List[List[str]], ground_paths: List[List[str]]):
    for prediction in predictions:
        for p in prediction:
            p = p.strip()
        if prediction in ground_paths:
            return False
    return True
    
def get_error_flag(prediction: List[str], answer: List[str]):
    for p in prediction:
        if p.strip() in answer:
            return False
    return True

def rules_error_analysis():
    id_list = []
    error_id_list = []
    retrieved_path_list = []
    ground_truth_path_list = []
    questions = []
    predictions = []
    answers = []
    
    # rog rules
    for line in open(rog_rules_file):
        line = json.loads(line)
        id_list.append(line['id'])
        if get_error_flag(line['prediction'], line['ground_paths']):
            error_id_list.append(line['id'])
            questions.append(line['question'])
            retrieved_path_list.append(line['prediction'])
            ground_truth_path_list.append(line['ground_paths'])

    # rog reasoning
    for line in open(rog_reasoning_predictions):
        line = json.loads(line)
        if line['id'] in error_id_list:
            predictions.append(line['prediction'])
            answers.append(line['ground_truth'])
            
    # save to file
    with open(f'error_analysis_{dataset}_rules_error.jsonl', 'w') as f:
        for i in range(len(error_id_list)):
            f.write(json.dumps({'id': error_id_list[i], 'question': questions[i], 'reasoning_paths': retrieved_path_list[i], 'ground_paths': ground_truth_path_list[i], 'prediction': predictions[i], 'answer': answers[i]}) + '\n')
    return error_id_list

def reasoning_error_analysis():
    id_list = []
    error_id_list = []
    retrieved_path_list = []
    ground_truth_path_list = []
    questions = []
    predictions = []
    answers = []
    
    # rog reasoning
    for line in open(rog_reasoning_predictions):
        line = json.loads(line)
        if get_error_flag(line['prediction'], line['ground_truth']):
            error_id_list.append(line['id'])
            predictions.append(line['prediction'])
            answers.append(line['ground_truth'])
            
    # rog rules
    for line in open(rog_rules_file):
        line = json.loads(line)
        if line['id'] in error_id_list:
            questions.append(line['question'])
            retrieved_path_list.append(line['prediction'])
            ground_truth_path_list.append(line['ground_paths'])

    # save to file
    with open(f'error_analysis_{dataset}_reasoning_error.jsonl', 'w') as f:
        for i in range(len(error_id_list)):
            f.write(json.dumps({'id': error_id_list[i], 'question': questions[i], 'reasoning_paths': retrieved_path_list[i], 'ground_paths': ground_truth_path_list[i], 'prediction': predictions[i], 'answer': answers[i]}) + '\n')
    return error_id_list

def rule_reasoning_error_analysis():
    id_list = []
    error_id_list = []
    retrieved_path_list = []
    ground_truth_path_list = []
    questions = []
    predictions = []
    answers = []
    
    temp_id_list = []
    
     # rog rules
    for line in open(rog_rules_file):
        line = json.loads(line)
        if get_error_flag(line['prediction'], line['ground_paths']):
            temp_id_list.append(line['id'])
            questions.append(line['question'])
            retrieved_path_list.append(line['prediction'])
            ground_truth_path_list.append(line['ground_paths'])
    
    # rog reasoning
    for line in open(rog_reasoning_predictions):
        line = json.loads(line)
        if line['id'] in temp_id_list:
            if get_error_flag(line['prediction'], line['ground_truth']):
                error_id_list.append(line['id'])
                predictions.append(line['prediction'])
                answers.append(line['ground_truth'])
            
    # save to file
    with open(f'error_analysis_{dataset}_both_error.jsonl', 'w') as f:
        for i in range(len(error_id_list)):
            f.write(json.dumps({'id': error_id_list[i], 'question': questions[i], 'reasoning_paths': retrieved_path_list[i], 'ground_paths': ground_truth_path_list[i], 'prediction': predictions[i], 'answer': answers[i]}) + '\n')
    return error_id_list
    

In [67]:
rules_error_ids = rules_error_analysis()
reasoning_error_ids = reasoning_error_analysis()
both_error_ids = rule_reasoning_error_analysis()

print(f'rules error: {len(rules_error_ids)}')
print(f'reasoning error: {len(reasoning_error_ids)}')
print(f'both error: {len(both_error_ids)}')

AttributeError: 'list' object has no attribute 'strip'

In [59]:
# error occured in rules but not in reasoning, the final answer is correct
id_list = []
dict_list = []

for id in rules_error_ids:
    if id not in both_error_ids and id not in reasoning_error_ids:
        id_list.append(id)
        
for line in open(f'error_analysis_{dataset}_rules_error.jsonl'):
    line = json.loads(line)
    for id in id_list:
        if line['id'] == id:
            print(line)
            dict_list.append(line)
            
with open(f'error_analysis_{dataset}_rules_error_only_final_answer_correct.jsonl', 'w') as f:
    for line in dict_list:
        f.write(json.dumps(line) + '\n')

{'id': 'WebQTest-172', 'question': 'what do you call members of the senate', 'reasoning_paths': [['government.government_office_or_title.governmental_body_if_any', 'common.topic.notable_types', 'common.topic.notable_types'], ['government.governmental_body.offices_positions', 'common.topic.notable_types', 'common.topic.notable_types'], ['government.government_position_held.governmental_body', 'government.government_position_held.office_position_or_title']], 'ground_paths': [], 'prediction': [' Senator', 'Senator'], 'answer': ['Senator', 'Senate majority leader', 'President pro tempore', 'President of the Senate', 'Chief of staff', 'State Senator']}
{'id': 'WebQTest-178', 'question': 'who was the leader of the us during wwii', 'reasoning_paths': [['event.speech_or_presentation.speech_topic', 'event.speech_or_presentation.speaker_s'], ['government.government_office_or_title.office_holders', 'government.government_position_held.office_holder'], ['military.military_conflict.commanders', 'mi

In [60]:
# error occured in reasoning but not in rules

id_list = []
dict_list = []

for id in reasoning_error_ids:
    if id not in both_error_ids and id not in rules_error_ids:
        id_list.append(id)
        
for line in open(f'error_analysis_{dataset}_reasoning_error.jsonl'):
    line = json.loads(line)
    for id in id_list:
        if line['id'] == id:
            print(line)
            dict_list.append(line)
            
with open(f'error_analysis_{dataset}_reasoning_error_only.jsonl', 'w') as f:
    for line in dict_list:
        f.write(json.dumps(line) + '\n')

{'id': 'WebQTest-1', 'question': 'what did james k polk do before he was president', 'reasoning_paths': [['government.government_position_held.office_holder', 'government.government_position_held.office_position_or_title'], ['people.person.profession', 'base.onephylogeny.type_of_thing.things_of_this_type'], ['people.person.nationality', 'government.government_office_or_title.jurisdiction']], 'ground_paths': [['government.government_position_held.office_holder', 'government.government_office_or_title.office_holders'], ['government.government_position_held.office_holder', 'government.government_position_held.office_position_or_title'], ['government.politician.government_positions_held', 'government.government_office_or_title.office_holders'], ['people.person.nationality', 'government.government_office_or_title.jurisdiction'], ['people.person.profession', 'base.onephylogeny.type_of_thing.things_of_this_type']], 'prediction': [' United States Representative'], 'answer': ['United States Rep

In [61]:
# error occured in both rules and reasoning
for id in both_error_ids:
    print(id)

WebQTest-3
WebQTest-9
WebQTest-12
WebQTest-28
WebQTest-31
WebQTest-32
WebQTest-34
WebQTest-36
WebQTest-38
WebQTest-39
WebQTest-46
WebQTest-59
WebQTest-69
WebQTest-71
WebQTest-76
WebQTest-77
WebQTest-106
WebQTest-107
WebQTest-114
WebQTest-115
WebQTest-123
WebQTest-124
WebQTest-134
WebQTest-141
WebQTest-142
WebQTest-155
WebQTest-209
WebQTest-221
WebQTest-231
WebQTest-241
WebQTest-245
WebQTest-273
WebQTest-277
WebQTest-289
WebQTest-317
WebQTest-337
WebQTest-359
WebQTest-360
WebQTest-379
WebQTest-383
WebQTest-385
WebQTest-386
WebQTest-390
WebQTest-406
WebQTest-421
WebQTest-472
WebQTest-479
WebQTest-482
WebQTest-488
WebQTest-489
WebQTest-497
WebQTest-504
WebQTest-521
WebQTest-532
WebQTest-535
WebQTest-546
WebQTest-550
WebQTest-556
WebQTest-561
WebQTest-564
WebQTest-568
WebQTest-583
WebQTest-592
WebQTest-603
WebQTest-609
WebQTest-614
WebQTest-616
WebQTest-619
WebQTest-620
WebQTest-622
WebQTest-624
WebQTest-628
WebQTest-631
WebQTest-639
WebQTest-648
WebQTest-663
WebQTest-664
WebQTest-667
WebQ